# TODO
* Build training and testing dataset - done
* Build json file for build and function to read and execute builds
  * cdd
* Add GLO

In [ ]:
import torch
import torch.nn as nn
import numpy as np

import os
import imageio
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from tqdm import tqdm

from dataset import get_rays
from rendering import rendering
from model import Voxels, Nerf
from ml_helpers import training

# Hyperparameters / Dataset

In [ ]:
SEED = 18
batch_size = 1024   # 4096 used in paper, 1024 used in code
nb_epochs = 25
lr = 1e-3   # 5e-4 used in paper, 1e-3 used in code
#final_lr = 5e-5
gamma = .5
#gamma = (final_lr / lr) ** (1 / (nb_epochs - 1))
nb_bins = 100   # 128 used in paper, 100 used in code
hres, vres = 800, 800
hres_wu_low, hres_wu_high = hres//4, 3*hres//4
vres_wu_low, vres_wu_high = vres//4, 3*vres//4

#dataset = 'fox'
dataset = 'helmet/_temp'
datapath = f'C:/_sw/eb_python/deep_learning/_dataset/NeRF/images/{dataset}'

torch.manual_seed(SEED)
np.random.seed(SEED)

train_o, train_d, target_px_values = get_rays(datapath, mode='train')
train_dataloader = DataLoader(torch.cat((torch.from_numpy(train_o).reshape(-1, 3).type(torch.float),
                                   torch.from_numpy(train_d).reshape(-1, 3).type(torch.float),
                                   torch.from_numpy(target_px_values).reshape(-1, 3).type(torch.float)), dim=1),
                                   batch_size=batch_size, shuffle=True)
train_dataloader_warmup = DataLoader(torch.cat((torch.from_numpy(train_o).reshape(-1, hres, vres, 3)[:, hres_wu_low:hres_wu_high, vres_wu_low:vres_wu_high, :].reshape(-1, 3).type(torch.float),
                               torch.from_numpy(train_d).reshape(-1, hres, vres, 3)[:, hres_wu_low:hres_wu_high, vres_wu_low:vres_wu_high, :].reshape(-1, 3).type(torch.float),
                               torch.from_numpy(target_px_values).reshape(-1, hres, vres, 3)[:, hres_wu_low:hres_wu_high, vres_wu_low:vres_wu_high, :].reshape(-1, 3).type(torch.float)), dim=1),
                               batch_size=batch_size, shuffle=True)

test_o, test_d, test_target_px_values = get_rays(datapath, mode='test')
#test_dataloader = DataLoader(torch.cat((torch.from_numpy(test_o).reshape(-1, 3).type(torch.float),
#                                   torch.from_numpy(test_d).reshape(-1, 3).type(torch.float),
#                                   torch.from_numpy(target_px_values).reshape(-1, 3).type(torch.float)), dim=1),
#                                   batch_size=batch_size, shuffle=True)

# Testing

In [ ]:
def mse2psnr(mse):
    return 20 * np.log10(1 / np.sqrt(mse))


@torch.no_grad()
def test(model, o, d, tn, tf, nb_bins=100, chunk_size=10, H=400, W=400, target=None):
    
    o = o.chunk(chunk_size)
    d = d.chunk(chunk_size)
    
    image = []
    for o_batch, d_batch in zip(o, d):
        img_batch = rendering(model, o_batch, d_batch, tn, tf, nb_bins=nb_bins, device=o_batch.device)
        image.append(img_batch) # N, 3
    image = torch.cat(image)
    image = image.reshape(H, W, 3).cpu().numpy()
    
    if target is not None:
        mse = ((image - target)**2).mean()
        psnr = mse2psnr(mse)
    
    if target is not None: 
        return image, mse, psnr
    else:
        return image

In [ ]:
def ShowTrainResults(save_path, training_loss, title, window_size=10):
    window = np.ones(window_size) / window_size
    moving_average = np.convolve(training_loss, window, 'valid')

    plt.plot(training_loss)
    plt.plot(np.arange(window_size - 1, len(training_loss)), moving_average, label='Moving Average', linestyle='--')

    plt.ylim(0.00001, 1)
    plt.grid(which='major', linestyle='-', linewidth='0.5', color='black')
    plt.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')
    plt.yscale('log')
    plt.xlabel('Batches')
    plt.ylabel('Training Loss')
    plt.title(f'{title}', fontsize=10)
    plt.legend()
    plt.savefig(f'{save_path}/results/{title}.png', dpi=300)  # Saves the plot as a PNG file with 300 DPI
    plt.show()

In [ ]:
def ShowTestImages(save_path, model, tn, tf, device, title, H, W):
    col, row = 4, 13
    fig, axs = plt.subplots(row, col, figsize=(20, 65))
    img_idx = 0
    for r in tqdm(range(row)):
        for c in range(col):
            #img_idx = 72 * r + 18 * c
            img, mse, psnr = test(model, torch.from_numpy(test_o[img_idx]).to(device).float(), torch.from_numpy(test_d[img_idx]).to(device).float(),
                    tn, tf, nb_bins=100, chunk_size=10, H=hres, W=vres, target=test_target_px_values[img_idx].reshape(H, W, 3))
            axs[r, c].imshow(img, cmap='gray')
            axs[r, c].set_title(f'Image {img_idx}, PSNR: {psnr:.1f}')
            axs[r, c].axis('off')  # Hide axis for a cleaner look
            img_idx += 1

    plt.tight_layout()  # Adjust subplots to fit in the figure area
    fig.suptitle(title, fontsize=16, y=1.05)
    plt.savefig(f'{save_path}/results/{title}.png', dpi=300)  # Saves the plot as a PNG file with 300 DPI
    plt.show()

# Training

In [ ]:
device = 'cuda'

#tn, tf = 8., 12.
tn, tf = 2., 6.

model = Nerf(hidden_dim=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5, 10], gamma=gamma)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

training_loss = training(model, optimizer, scheduler, tn, tf, nb_bins, 1, train_dataloader_warmup, datapath, 'warmup', device=device)
ShowTrainResults(datapath, training_loss, "Training Loss for warmup", window_size=10)
ShowTestImages(datapath, model, tn, tf, device, "Test images for wamrup dataloader", hres, vres)

training_loss = training(model, optimizer, scheduler, tn, tf, nb_bins, nb_epochs, train_dataloader, datapath, 'epoch', device=device)
ShowTrainResults(datapath, training_loss, f'Training Loss for epoch #{nb_epochs}', window_size=600)
ShowTestImages(datapath, model, tn, tf, device, f'Test images for epoch #{nb_epochs}', hres, vres)